# **Physics Engine**

**Imports**

In [581]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random

from IPython.display import HTML
from math import sin, cos, radians, atan2, sqrt, pi
from matplotlib.patches import Polygon
from matplotlib import animation
from typing import Final

**Global variables**

In [582]:
x: Final[int] = 0
y: Final[int] = 1

interval: Final[int] = 10               # (ms) frame interval for animation
dt: Final[float] = interval / 1000      # (s) delta time
time = 0                                # (s) time

g: Final[list] = np.array([0, -9.81])   # (m/s^2) gravity
e: Final[float] = 0.8                   # restitution coefficient

normal = []

**Polygon data handling**

In [583]:
class PolyData:

    verteces = []           # x and y coordinates of the verteces relative to the center of mass
    center = []             # x and y coordinates of the center of mass relative to the origin
    speed = 0               # (m/s) speed of the polygon
    angle = 0               # (degrees) angle of the speed
    velocity = []           # (m/s) x and y components of the velocity
    spin = 0                # (rad/s) spin of the polygon
    mass = 0                # (kg) mass of the polygon
    inertia = 0             # (kg*m^2) moment of inertia of the polygon from: https://www.omnicalculator.com/physics/mass-moment-of-inertia & https://www.omnicalculator.com/math/trigonometry
    collider = []           # x and y coordinates of the collision vertex relative to the center of mass
    colliderVelocity = []   # (m/s) x and y components of the velocity of the collision vertex
    impulse = None          # (N*s) impulse of the collision

    def __init__(self, verteces, center, speed, angle, spin, mass, inertia):
        self.verteces = verteces
        self.center = center
        self.speed = speed
        self.angle = angle
        self.spin = spin
        self.mass = mass
        self.inertia = inertia
        self.velocity = np.array([
            self.speed * cos(radians(self.angle)),
            self.speed * sin(radians(self.angle))])

# Calculate the speed of the polygon based on its velocity
    def updateSpeed(self):
        self.speed = sqrt(self.velocity[x]**2 + self.velocity[y]**2)

# Calculate the angle of the speed based on the velocity
    def updateAngle(self):
        self.angle = atan2(self.velocity[y], self.velocity[x]) * 180/pi

# Calculate the velocity of the polygon
    def updateVelocity(self):
        # if there is an impulse, use it to calculate the velocity
        if (self.impulse):
            self.velocity += (self.impulse / self.mass) * normal
        # if there is no impulse, use gravity to calculate the velocity
        else:
            self.velocity += g * dt

# Calculate the spin speed of the polygon
    def updateSpin(self):
        self.spin += (self.impulse / self.inertia) * np.cross(self.collider, normal)

# Calculate the velocity of the collision vertex
    def updateColliderVelocity(self):
        self.colliderVelocity = np.array([
            self.velocity[x] - self.spin * self.collider[y],
            self.velocity[y] + self.spin * self.collider[x]
        ])

# Calculate the impulse of a collision with a line
    def updateImpulse(self):
        self.impulse = -(1 + e) * (np.dot(self.colliderVelocity, normal) / \
                            (1 / self.mass + np.cross(self.collider, normal)**2 / self.inertia))
            
# Calculate a new position for the polygon
    def updatePosition(self):
        self.center += self.velocity * dt

# Calculate a new rotation for the polygon
    def updateRotation(self):
        # calculate the rotation matrix
        theta = self.spin * dt

        rotation = np.array((
            (cos(theta), -sin(theta)),
            (sin(theta),  cos(theta))))

        verteces = range(len(self.verteces))

        # rotate the verteces around the center of mass
        for vertex in verteces:
            self.verteces[vertex, :] = np.dot(rotation, self.verteces[vertex, :])

**Collision detection**

In [584]:
def collision(index):
    global normal, polygons, lines

    # Get the polygon of interest
    polygon = polygons[index]

    # For each line in the list of lines
    for line in lines:
        # Calculate the slope and intercept of the line
        vector = [line[1][x] - line[0][x], line[1][y] - line[0][y]]
        slope = vector[y] / vector[x]
        intercept = line[0][y] - slope * line[0][x]

        # For each vertex in the polygon
        for i in range(len(polygon.verteces)):
            vertex = polygon.center + polygon.verteces[i]

            # Determine if the vertex is above or below the line
            if (vertex[y] < slope * vertex[x] + intercept):
                
                # If the vertex is below the line, set it as the collider
                polygon.collider = polygon.verteces[i]
                polygon.updateColliderVelocity()

                # Calculate the normal vector from the line
                absolute = sqrt(vector[x]**2 + vector[y]**2)
                normal = np.array([-vector[y], vector[x]]) / absolute

                # Determine if the polygon is moving towards the line
                if (np.dot(polygon.colliderVelocity, normal) < 0):
                    return True

    # Copy the list of polygons and remove the polygon of interest
    otherPolygons = polygons.copy()
    otherPolygons.pop(index)

    # Get the verteces of the polygon of interest
    verteces = polygon.verteces + polygon.center
        
    # For each polygon in the list of other polygons (excluding the polygon of interest)
    for otherPolygon in otherPolygons:
        # Get the verteces of the other polygon
        otherVerteces = otherPolygon.verteces + otherPolygon.center

        # For each vertex in the other polygon 
        for j in range(len(otherVerteces)):
            count = 0
            closest = 0
            parallel = []

            # For each vertex in the polygon of interest
            for i in range(len(verteces)):
                # If the vertex is the last in the list, get the vector to the first vertex
                if (i == len(verteces) - 1):
                    i = -1

                # Calculate the vector from the vertex to the next vertex
                vector = verteces[i + 1] - verteces[i]
                # Calculate the vector from the vertex to the foreign vertex
                foreignVector = otherVerteces[j] - verteces[i]
                
                # Assign the closest vector to the foreign vertex as the collision vector
                if (abs(np.cross(vector, foreignVector)) <= closest or closest == 0):
                    closest = abs(np.cross(vector, foreignVector))
                    parallel = vector

                # Determine if the foreign vertex is to the left of the vector
                if (np.cross(vector, foreignVector) > 0):
                    count += 1

            # If the foreign vertex is inside the polygon
            if (count == len(verteces)):

                # Set the foreign vertex as the collider for both polygons and update their velocities
                polygon.collider = otherVerteces[j] - polygon.center
                otherPolygon.collider = otherPolygon.verteces[j]
                polygon.updateColliderVelocity()
                otherPolygon.updateColliderVelocity()

                # Calculate the normal vector from the collision vector
                absolute = sqrt(parallel[x]**2 + parallel[y]**2)
                normal = np.array([parallel[y], -parallel[x]]) / absolute

                # If the polygons are moving towards each other
                if (np.dot(otherPolygon.colliderVelocity - polygon.colliderVelocity, normal) < 0):
                    # Calculate the impulse of the collision
                    impulse = -(1 + e) * (np.dot(otherPolygon.colliderVelocity - polygon.colliderVelocity, normal) / \
                                    (1 / otherPolygon.mass + 1 / polygon.mass + np.cross(otherPolygon.collider, normal)**2 / \
                                    otherPolygon.inertia + np.cross(polygon.collider, normal)**2 / polygon.inertia))

                    # Update the impulse and spin for both polygons
                    polygon.impulse = -impulse
                    otherPolygon.impulse = impulse
                    polygon.updateSpin()
                    otherPolygon.updateSpin()
                    
                    return False
    return False

**Initialization & animation**

In [585]:
# initialize figure and axes
fig, ax = plt.subplots(figsize=(6, 3), dpi=254)

# set style, axes limits, and labels
plt.style.use('dark_background')
plt.xlim(0, 128)
plt.ylim(-5, 59)
plt.title('x (m)', size=7)
plt.ylabel('y (m)', size=7)
plt.tick_params(axis='both', which='major', labelsize=7)

# initialize line and trace plots
line1, = ax.plot([], color='white', linewidth=0.75, zorder=2)
line2, = ax.plot([], color='white', linewidth=0.75, zorder=2)
trace1, = ax.plot([], color='grey', linewidth=0.75, linestyle='dotted', zorder=0)
trace2, = ax.plot([], color='grey', linewidth=0.75, linestyle='dotted', zorder=0)
trace3, = ax.plot([], color='grey', linewidth=0.75, linestyle='dotted', zorder=0)

# initialize line and polygon data
wallL = np.array([
    [0, 59], 
    [0.0000000000001, 0]])

wallR = np.array([
    [128, 0], 
    [128.0000000000001, 59]])

ground = np.array([
    [ 0, 0], 
    [63.68, 0]])

incline = np.array([
    [ 64, 0], 
    [128, 32]])

square = PolyData(np.array([
    [-1., -1.],
    [ 1., -1.],
    [ 1.,  1.],
    [-1.,  1.]
]), np.array([1., random.uniform(5, 10)]), random.uniform(20, 30), random.uniform(35, 65), random.uniform(-10, 10), 100, 66.67)

triangle = PolyData(np.array([
    [-1., -1.],
    [ 1., -1.],
    [ 0.,  4.]
]), np.array([127., random.uniform(42, 47)]), random.uniform(20, 30), random.uniform(-35 - 90, -65 - 90), random.uniform(-5, 5), 100, 1171.8)

hexagon = PolyData(np.array([
    [-1.5, 0],
    [-0.75, -3*sqrt(3)/4],
    [ 0.75, -3*sqrt(3)/4],
    [ 1.5, 0],
    [ 0.75,  3*sqrt(3)/4],
    [-0.75,  3*sqrt(3)/4]
]), np.array([75., 57.5]), 0, 0, random.uniform(-10, 10), 100, 41.67)

# initialize lists
polygons = [square, triangle, hexagon]
lines = [ground, incline, wallL, wallR]
traceList1 = []
traceList2 = []
traceList3 = []

# initialize polygon plots
squareVerteces = Polygon(square.verteces)
triangleVerteces = Polygon(triangle.verteces)
hexagonVerteces = Polygon(hexagon.verteces)
squareVerteces.set_color('c')
triangleVerteces.set_color('m')
hexagonVerteces.set_color('y')
squareVerteces.set_zorder(1)
triangleVerteces.set_zorder(1)
hexagonVerteces.set_zorder(1)
ax.add_patch(squareVerteces)
ax.add_patch(triangleVerteces)
ax.add_patch(hexagonVerteces)

# initialize plots
def init():
    global traceList1, traceList2, traceList3
    line1.set_data(ground[:, x], ground[:, y])
    line2.set_data(incline[:, x], incline[:, y])

    squareVerteces.set_xy(square.verteces + square.center)
    triangleVerteces.set_xy(triangle.verteces + triangle.center)
    hexagonVerteces.set_xy(hexagon.verteces + hexagon.center)

    traceList1 = np.array([square.center])
    traceList2 = np.array([triangle.center])
    traceList3 = np.array([hexagon.center])

    trace1.set_data(traceList1[:, x], traceList1[:, y])
    trace2.set_data(traceList2[:, x], traceList2[:, y])
    trace3.set_data(traceList3[:, x], traceList3[:, y])

# animation function loop
def animate(f):
    global time, traceList1, traceList2, traceList3
    # update time
    time += dt

    # iterate through the physics calculations for each polygon
    for i, polygon in enumerate(polygons):

        if (collision(i)):
            polygon.updateImpulse()
            polygon.updateSpin()
            
        polygon.updateVelocity()
        polygon.updatePosition()
        polygon.updateRotation()
        polygon.impulse = None

    squareVerteces.set_xy(square.verteces + square.center)
    triangleVerteces.set_xy(triangle.verteces + triangle.center)
    hexagonVerteces.set_xy(hexagon.verteces + hexagon.center)

    traceList1 = np.append(traceList1, [square.center], axis=0)
    traceList2 = np.append(traceList2, [triangle.center], axis=0)
    traceList3 = np.append(traceList3, [hexagon.center], axis=0)

    trace1.set_data(traceList1[:, x], traceList1[:, y])
    trace2.set_data(traceList2[:, x], traceList2[:, y])
    trace3.set_data(traceList3[:, x], traceList3[:, y])

# animation initialization
anim = animation.FuncAnimation(
    fig, animate, init_func=init, frames=round(15000 / interval), interval=interval)

# export animation as html5 video
video = anim.to_html5_video()
html = HTML(video)
display(html)
plt.close()